In [2]:
# Import libraries

from bs4 import BeautifulSoup
import urllib.request as ureq
from urllib.request import urlopen
import urllib.parse, urllib.error
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import pandas as pd
import numpy as np
import time

In [5]:
searched_words = ['Data-Scientist-jobs','Data-Science-jobs','Data-Analyst-jobs']
indeed_url = 'https://www.seek.com.au/'
pages = list(range(1,10))

# Concatenate base_url, searched word values and page numbers to define the url for all pages

searched_url_all = [str(indeed_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) for i in pages for words in searched_words]
searched_url_150k = [str(indeed_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) + '&salaryrange=150000-999999&salarytype=annual' for i in pages for words in searched_words]
searched_url_100k = [str(indeed_url) + words + '/in-All-Melbourne-VIC?page=' + str(i) + '&salaryrange=100000-999999&salarytype=annual' for i in pages for words in searched_words]


In [6]:
# Scrape searched urls and save as BeautifulSoup objects

soup_objects = []

for urls in searched_url_100k:
    soup = BeautifulSoup(ureq.urlopen(urls), "lxml")
    soup_objects.append(soup)
    
# Extract all job ids and append to job_ids

job_ids = []

for soup in soup_objects:
    job_id = Selector(text=soup.prettify()).xpath('//article/@data-job-id').extract()
    
    for ids in job_id:
        job_ids.append(ids)
len(job_ids)

# Extract individual urls for each job_id

job_base_url = 'https://www.seek.com.au/job/'

# Concatenate base_url and job_id values to define the url for each job ad

job_id_url = [job_base_url + x for x in job_ids]

len(job_id_url)

# Extract soup objects from each job_id

job_soup_objects = []

for job_urls in job_id_url:
    soups = BeautifulSoup(ureq.urlopen(job_urls), "lxml")
    job_soup_objects.append(soups)
    time.sleep(1)

len(job_soup_objects)

232

In [7]:
# Extract required elements from all the individual job_id_url

job_advertiser_list = []
job_title_list = []
job_class_list = []
job_sub_class_list = []
job_type_list = []
job_location_list = []
job_post_date_list = []
job_salary_list = []
job_desc_list = []

for job_soup in job_soup_objects:
    
    ext_job_advertiser = Selector(text=job_soup.prettify()).xpath('//div/h2/span[2]/span/text()').extract()
    job_advertiser_list.append(ext_job_advertiser)
    
    ext_job_title = Selector(text=job_soup.prettify()).xpath('//span/span/h1/text()').extract()
    job_title_list.append(ext_job_title)
    
    ext_job_class = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div[2]/dd/span/span/strong/text()').extract()
    job_class_list.append(ext_job_class)
    
    ext_job_sub_class = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div/dd/span/span/span/text()').extract()
    job_sub_class_list.append(ext_job_sub_class)
    
    ext_job_type = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/dd[3]/span/span').extract()
    job_type_list.append(ext_job_type)
    
    ext_job_location = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/dd[2]/span/span/strong').extract()
    job_location_list.append(ext_job_location)
    
    ext_job_post_date = Selector(text=job_soup.prettify()).xpath('//article/div/div[2]/div/section/dl/dd[1]/span/span').extract()
    job_post_date_list.append(ext_job_post_date)
    
    ext_job_salary = Selector(text=job_soup.prettify()).xpath('//aside/span/div[2]/div/section/dl/div[1]/dd/span/span').extract()
    job_salary_list.append(ext_job_salary)
    
    ext_job_desc = Selector(text=job_soup.prettify()).xpath('//article/div/div/div/span/div/div/div/div/div').extract()
    job_desc_list.append(ext_job_desc)
    
print(len(job_advertiser_list))
print(len(job_title_list))
print(len(job_class_list))
print(len(job_sub_class_list))
print(len(job_type_list))
print(len(job_location_list))
print(len(job_post_date_list))
print(len(job_salary_list))
print(len(job_desc_list))


232
232
232
232
232
232
232
232
232


In [8]:
 #Create DataFrame for all, med, high searches

med_df = pd.DataFrame(job_ids, columns =['job_ids'])
med_df['job_advertiser'] = job_advertiser_list
med_df['job_title'] = job_title_list
med_df['job_class'] = job_class_list
med_df['job_sub_class'] = job_sub_class_list
med_df['job_type'] = job_type_list
med_df['job_location'] = job_location_list
med_df['job_post_date'] = job_post_date_list
med_df['job_salary'] = job_salary_list
med_df['job_desc'] = job_desc_list

In [9]:
med_df.head(2)

,job_ids,job_advertiser,job_title,job_class,job_sub_class,job_type,job_location,job_post_date,job_salary,job_desc
0,36938016,[\n SEEK\n ],[\n Software Engineer\n ...,[\n Information & Communic...,"[\n , \n ...","[<span class=""lwHBT6d"">\n Fu...","[<strong class=""lwHBT6d"">\n ...","[<span class=""lwHBT6d"">\n 1...","[<span class=""lwHBT6d"">\n B...","[<div class=""templatetext"">\n ..."
1,37041659,[\n Xpand Group Pty Ltd\n ...,[\n Data Scientist\n ...,[],"[\n , \n ...","[<span class=""lwHBT6d"">\n Co...","[<strong class=""lwHBT6d"">\n ...","[<span class=""lwHBT6d"">\n 2...","[<span class="""">\n <strong ...","[<div class=""templatetext"">\n ..."


In [13]:
#Export dataframe to csv
#filepath = 'jobs_high.csv'
#high_df.to_csv(filepath, index=False)

In [14]:
#Export dataframe to csv
#filepath = 'jobs_all.csv'
#all_df.to_csv(filepath, index=False)

In [37]:
#Export dataframe to csv
filepath = 'jobs_med.csv'
med_df.to_csv(filepath, index=False)

In [38]:
import pandas as pd
all_df_csv = 'jobs_med.csv'
all_df = pd.read_csv(all_df_csv, encoding='iso-8859-1', index_col ='job_ids')

In [18]:
all_df.shape

(232, 9)

# Resubmit Part

# Cleaning  And EDA

In [39]:
# Clean job_advertiser

newAdvertisers=[]

for a in all_df.job_advertiser:
    try:
        s=a.split('\\n')[1].lstrip()
        #print(s)
        newAdvertisers.append(s)
    except:
        newAdvertisers.append('')
        
newAdvertisers[0:5]

['SEEK',
 'Xpand Group Pty Ltd',
 'Treasury Wine Estates',
 'FourQuarters Recruitment',
 'Greythorn Experis']

In [40]:
 #Clean job_title

new_job_title = []

for b in all_df.job_title:
    try:
        t=b.split('\\n')[1].lstrip()
        
        new_job_title.append(t)
    except:
        new_job_title.append('')
        
new_job_title[0:5]

['Software Engineer',
 'Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Senior Data Scientist']

In [22]:
# Clean job_class

new_job_class = []

for c in all_df.job_class:
    try:
        u=c.split('\\n')[1].lstrip()
        
        new_job_class.append(u)
    except:
        new_job_class.append('')
        
new_job_class[0:5]

['Information & Communication Technology',
 '',
 '',
 '',
 'Information & Communication Technology']

In [23]:
# Clean job_sub_class

new_job_sub_class = []

for d in all_df.job_sub_class:
    try:
        v=d.split('\\n')[2].lstrip()
        
        new_job_sub_class.append(v)
    except:
        new_job_sub_class.append('')
        
new_job_sub_class[0:5]

['Engineering - Software',
 'Other',
 'Mathematics, Statistics & Information Sciences',
 'Consultants',
 'Database Development & Administration']

In [24]:
# Clean job_type

new_job_type = []

for e in all_df.job_type:
    try:
        u=e.split('\\n')[1].lstrip()
        
        new_job_type.append(u)
    except:
        new_job_type.append('')
        
new_job_type[0:5]

['Full Time', 'Contract/Temp', 'Full Time', 'Full Time', 'Full Time']

In [43]:
# Add new salary column and populate values with '150k+' or '100-150k' or below_100k' to DataFrame
all_df['salary'] = 'below_100k'

# Add new cleaned newAdvertisers column to DataFrame
all_df['advertiser'] = newAdvertisers

# Add new cleaned new_job_title column to DataFrame
all_df['title'] = new_job_title

# Add new cleaned new_job_class column to DataFrame
all_df['class'] = new_job_class

# Add new cleaned new_job_sub_class column to DataFrame
all_df['sub_class'] = new_job_sub_class

# Add new cleaned new_job_type column to DataFrame
all_df['type'] = new_job_type

#all_df['job_desc'] = new_job_desc

In [44]:
all_df.columns

Index(['job_advertiser', 'job_title', 'job_class', 'job_sub_class', 'job_type',
       'job_location', 'job_post_date', 'job_salary', 'job_desc', 'salary',
       'advertiser', 'title', 'class', 'sub_class', 'type'],
      dtype='object')

In [27]:
 #Drop columns that are no longer required from DataFrame

#all_df.drop(all_df.iloc[:, 0:9], inplace=True, axis=1)

In [45]:
# Inspect columns and shape for all_df

print(all_df.columns)
print(all_df.shape)
all_df.head(1)

Index(['job_advertiser', 'job_title', 'job_class', 'job_sub_class', 'job_type',
       'job_location', 'job_post_date', 'job_salary', 'job_desc', 'salary',
       'advertiser', 'title', 'class', 'sub_class', 'type'],
      dtype='object')
(232, 15)


,job_advertiser,job_title,job_class,job_sub_class,job_type,job_location,job_post_date,job_salary,job_desc,salary,advertiser,title,class,sub_class,type
job_ids,,,,,,,,,,,,,,,
36938016,['\n SEEK\n '],['\n Software Engineer\n ...,['\n Information & Communi...,"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<div class=""templatetext"">\n ...",below_100k,SEEK,Software Engineer,Information & Communication Technology,Engineering - Software,Full Time


In [46]:


all_df.drop_duplicates(keep='first', inplace=True)

In [47]:
all_df.columns

Index(['job_advertiser', 'job_title', 'job_class', 'job_sub_class', 'job_type',
       'job_location', 'job_post_date', 'job_salary', 'job_desc', 'salary',
       'advertiser', 'title', 'class', 'sub_class', 'type'],
      dtype='object')

In [48]:
all_df

,job_advertiser,job_title,job_class,job_sub_class,job_type,job_location,job_post_date,job_salary,job_desc,salary,advertiser,title,class,sub_class,type
job_ids,,,,,,,,,,,,,,,
36938016,['\n SEEK\n '],['\n Software Engineer\n ...,['\n Information & Communi...,"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<div class=""templatetext"">\n ...",below_100k,SEEK,Software Engineer,Information & Communication Technology,Engineering - Software,Full Time
37041659,['\n Xpand Group Pty Ltd\n ...,['\n Data Scientist\n ...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n C...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,Xpand Group Pty Ltd,Data Scientist,,Other,Contract/Temp
37038983,['\n Treasury Wine Estates\n...,['\n Data Scientist\n ...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,Treasury Wine Estates,Data Scientist,,"Mathematics, Statistics & Information Sciences",Full Time
37023029,['\n FourQuarters Recruitmen...,['\n Data Scientist\n ...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,FourQuarters Recruitment,Data Scientist,,Consultants,Full Time
37022915,['\n Greythorn Experis\n ...,['\n Senior Data Scientist\n...,['\n Information & Communi...,"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<div class=""templatetext"">\n ...",below_100k,Greythorn Experis,Senior Data Scientist,Information & Communication Technology,Database Development & Administration,Full Time
37022253,['\n Greythorn Experis\n ...,['\n Senior Data Scientist -...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n C...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,Greythorn Experis,Senior Data Scientist - 12 month contract,,Other,Contract/Temp
37012533,['\n Xpand Group Pty Ltd\n ...,['\n Senior Data Scientist\n...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,Xpand Group Pty Ltd,Senior Data Scientist,,Other,Full Time
36998552,['\n Talentpool Recruitment\...,['\n Data Scientist - $750 -...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n C...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,Talentpool Recruitment,Data Scientist - $750 - $900/day,,Developers/Programmers,Contract/Temp
36963272,['\n FourQuarters Recruitmen...,['\n Data Scientist\n ...,[],"['\n ', '\n ...","['<span class=""lwHBT6d"">\n F...","['<strong class=""lwHBT6d"">\n ...","['<span class=""lwHBT6d"">\n ...","['<span class="""">\n <strong...","['<div class=""templatetext"">\n ...",below_100k,FourQuarters Recruitment,Data Scientist,,Consultants,Full Time


In [50]:
# Clean jobs_desc column 

all_df.job_desc = all_df.job_desc.str.replace('[0-9]','') # removes digits
all_df.job_desc = all_df.job_desc.str.replace('/_/','') # removes words that starts with underscore _

In [52]:
# Create new DataFrame and dummify required categorical variables (except for job_desc and salary)
jobs_dummy = pd.get_dummies(all_df, columns = ['advertiser', 'class','sub_class', 'title','type'], drop_first = True)

In [53]:
# View salary values
jobs_dummy['salary'].unique()

array(['below_100k'], dtype=object)

In [54]:
jobs_dummy.insert(len(jobs_dummy), 'salary_range', 0)

In [55]:

jobs_dummy.loc[jobs_dummy['salary'] == '100-150k', 'salary_range'] = 1
jobs_dummy.loc[jobs_dummy['salary'] == '150k+', 'salary_range'] = 2       

# Print the count of each salary values
jobs_dummy['salary_range'].value_counts()

0    182
Name: salary_range, dtype: int64

# Modelling

In [56]:
y = jobs_dummy['salary_range']

In [57]:
# Define X as all variables except for salary and salary_range

X = jobs_dummy
X.drop(['salary', 'salary_range'], axis=1, inplace=True)
X.columns

Index(['job_advertiser', 'job_title', 'job_class', 'job_sub_class', 'job_type',
       'job_location', 'job_post_date', 'job_salary', 'job_desc',
       'advertiser_7 Eleven',
       ...
       'title_Systems Finance Analyst - SAP - Immediately Available',
       'title_Technical Business Analyst - Data',
       'title_Technical Business Analyst - Data Migration & Reporting',
       'title_Technical Business Analyst / Data Analyst',
       'title_Technical Lead - Data Platform',
       'title_Technical Project Manager', 'title_Technology & Data Director',
       'title_Test Analyst - Data Analytics - Melbourne', 'type_Full Time',
       'type_Part Time'],
      dtype='object', length=299)

In [58]:
# Split jobs_dummy to train and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(121, 299) (121,)
(61, 299) (61,)


In [59]:
# Use TfidfVectorizer to extract features from jobs_desc column and fit to train (this is to be used only if vectorized jobs_desc is required for model)

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialise TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', token_pattern= '\w+')

# Fit and transform the training data

tfidf_train = tfidf_vectorizer.fit_transform(X_train['job_desc'])
print("train data " + str(tfidf_train.shape))

# Fit and transform the test data

tfidf_test = tfidf_vectorizer.transform(X_test['job_desc'])
print("test data " + str(tfidf_test.shape))

train data (121, 3896)
test data (61, 3896)


In [60]:
# Get the feature names of TfidfVectorizer (this is to be used only if vectorized jobs_desc is required for model))

tfidf_features = tfidf_vectorizer.get_feature_names()
print(tfidf_features)

['__base', '__top', '_b', '_b_base', '_blank', '_brkmzddy', '_h_header', '_hhdni', '_lqcj', 'aaai', 'abap', 'abilities', 'ability', 'able', 'aboriginal', 'absolute', 'absorbing', 'academic', 'academics', 'accelerates', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'access', 'accessibility', 'accessible', 'accommodate', 'accomplish', 'accomplished', 'accordance', 'according', 'account', 'accountabilities', 'accountability', 'accountable', 'accounting', 'accounts', 'accuracy', 'accurate', 'accurately', 'achieve', 'achievement', 'achievements', 'achieving', 'acknowledged', 'acquire', 'acquired', 'acquisition', 'acquisitions', 'acrobat', 'act', 'action', 'actionable', 'actions', 'active', 'actively', 'activities', 'activity', 'acts', 'actuarial', 'acumen', 'ad', 'adapt', 'adapting', 'add', 'added', 'addition', 'additional', 'additionally', 'address', 'addressed', 'addresses', 'addressing', 'adept', 'adherence', 'adjusted', 'adjustments', 'administer', 'administering', 'adm

In [61]:
# Remove job_desc
X_train = X_train.drop('job_desc', axis=1)
X_test = X_test.drop('job_desc', axis=1)

In [62]:
# Convert tfidf_train and tfidf_test to DataFrames (this is to be used only if vectorized jobs_desc is required for model)

df_tfidf_train = pd.DataFrame(tfidf_train.toarray(), columns = tfidf_features)
df_tfidf_test = pd.DataFrame(tfidf_test.toarray(), columns = tfidf_features)

In [63]:
print(X_train.shape)
print(X_test.shape)

(121, 298)
(61, 298)


In [64]:
# Replace with NaN values with zero (this is to be used only if vectorized jobs_desc is required for model))

X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
# Predict using Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

clf_dt = DecisionTreeClassifier() # Initiate Decision Tree as the classifier model

clf_dt.fit(X_train, y_train) # Fit model to the train set

y_pred = clf_dt.predict(X_test) # Predict test set

accuracy_score(y_test, y_pred) # Evaluate test-set accuracy

0.5